In [12]:
import os
import openai
import re
import requests
import json

In [2]:
# Load your openai API key
openai.api_key = os.getenv('OPENAI_API_KEY')

In [7]:
# Compile the regular expressions
english_word_regex = re.compile(r"English: (.*?)\nGerman")
german_word_regex = re.compile(r"\nGerman: (.*?)\nExample sentence")
english_example_regex = re.compile(r"\nExample sentence: (.*?)\nTranslation")
german_example_regex = re.compile(r"\nTranslation: (.+)$")

In [8]:
def create_english_german_flashcard(word):
    """
    Creates an English-German flashcard.

    :param word: English word or phrase to create a flashcard for.
    :return: A tuple containing the front and back content of the flashcard.
    :raises: Exception if the API call fails or the expected content is not found.
    """
    content = f"Create a single English-German index card for \"{word}\" with the translation of the word and one example sentence using the following structure: \nEnglish: English word/phrase\nGerman: German translation\nExample sentence: English sentence \nTranslation: German Sentence"
    
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": content}
    ]

    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages,
            temperature=0.7,
            max_tokens=500
        )
    except Exception as e:
        raise Exception(f"Failed to create flashcard due to an API error: {str(e)}")

    generated_flashcards = response.get("choices", [{}])[0].get("message", {}).get("content", "")

    try:
        english_word_match = english_word_regex.findall(generated_flashcards)[0]
        german_word_match = german_word_regex.findall(generated_flashcards)[0]
        english_example_match = english_example_regex.findall(generated_flashcards)[0]
        german_example_match = german_example_regex.findall(generated_flashcards)[0]
    except IndexError as e:
        raise Exception("Failed to parse the flashcard content.")

    front = f"English: {english_word_match}<div><br></div><div>Example sentence: {english_example_match}"
    back = f"German: {german_word_match}<div><br></div><div>Example sentence: {german_example_match}"

    return front, back


In [13]:
# Constants for the local Anki Wndows App API
API_URL = 'http://localhost:8765'
API_VERSION = 6

In [14]:
class APIError(Exception):
    """Custom exception class for API errors"""
    pass

def request(action, **params):
    """
    Create a request dictionary with the given action and parameters.

    :param action: The action to perform.
    :param params: Additional parameters for the action.
    :return: A dictionary representing the request.
    """
    return {'action': action, 'params': params, 'version': API_VERSION}

def invoke(action, **params):
    """
    Send a request to the API and return the result.

    :param action: The action to perform.
    :param params: Additional parameters for the action.
    :return: The result from the API.
    :raises APIError: If the API response contains an error.
    """
    request_json = json.dumps(request(action, **params)).encode('utf-8')
    response = requests.post(API_URL, request_json)
    response_json = response.json()

    if not isinstance(response_json, dict) or 'error' not in response_json or 'result' not in response_json:
        raise APIError('Response has an unexpected format or missing fields')

    if response_json['error'] is not None:
        raise APIError(response_json['error'])

    return response_json['result']

def add_note(deck, model, fields):
    """
    Add a note to a deck with the specified model and fields.

    :param deck: The name of the deck.
    :param model: The name of the model.
    :param fields: The fields for the note.
    :return: The result from the API.
    """
    note = {
        'deckName': deck,
        'modelName': model,
        'fields': fields,
        'options': {
            'allowDuplicate': False
        },
        'tags': []
    }
    return invoke('addNote', note=note)


In [18]:
# List of english words to which a flashcard should be created
list_words = ['to dispatch',
              'vast',
              'the fate',
              'incense',
              'strummed',
              'lute',
              'to itch',
              'a seal'
              ]

In [17]:
for word in list_words:
    front, back = create_card(word)
    deck_name = 'English Advanced'
    model_name = 'Basic (and reversed card)'
    fields = {
        'Front': front,
        'Back': back
    }
    print(front)
    result = add_note(deck_name, model_name, fields)
    if result is None:
        print('Note already exists')
    else:
        print('Note added with id', result)

English: to dispatch<div><br></div><div>Example sentence: The package was dispatched yesterday.
Note added with id 1705765855680
English: Vast<div><br></div><div>Example sentence: The vast ocean stretched out before us.
Note added with id 1705765860781
English: the fate<div><br></div><div>Example sentence: She believed in the power of fate to guide her life.
Note added with id 1705765866403
English: incense<div><br></div><div>Example sentence: I enjoy burning incense to create a relaxing atmosphere.
Note added with id 1705765872387
English: strummed<div><br></div><div>Example sentence: She strummed the guitar strings to create a soothing melody.
Note added with id 1705765878894
English: lute<div><br></div><div>Example sentence: He played the lute beautifully.
Note added with id 1705765883805
English: itch<div><br></div><div>Example sentence: My mosquito bite is starting to itch.
Note added with id 1705765888570
English: a seal<div><br></div><div>Example sentence: The seal was basking i